In [2]:
import json
import pprint

In [1]:
import os
git_dir = os.path.dirname(os.getcwd())
project_dir = os.path.join(git_dir,'zavmo')
os.chdir(project_dir)

In [3]:
import json
from helpers.chat import get_openai_completion,  get_prompt, make_structured_call, summarize_history, summarize_profile,create_message_payload,force_tool_call
from helpers.functions import parse_type
from IPython.display import Markdown
import openai
from openai import OpenAI
from helpers.functions import create_pydantic_model, get_yaml_data, create_system_message
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [106]:
stage_config = get_yaml_data('discuss')

In [5]:
resp_schema = get_yaml_data('common')['response']

In [6]:
resp_model    = create_pydantic_model(name=resp_schema['title'],
                                       description=resp_schema['description'],
                                       fields=resp_schema['fields']
                                       )
    
probe_system_message = create_system_message('discuss', stage_config, mode='probe')

In [7]:
required_fields = [f['title'] for f in stage_config['fields']]
required_fields

['interest_areas',
 'learning_style',
 'curriculum',
 'timeline',
 'goals_alignment']

In [8]:
profile_summary = """
Name: Mumtaz
Occupation: Junior Software Engineer

As a Junior Software Engineer, I am passionate about leveraging technology to solve complex problems and improve user experiences. I have a strong foundation in software development principles and a keen interest in expanding my knowledge and skills in various domains.

Learning Interests:
I am particularly interested in the following areas:

Statistics: Understanding data analysis techniques to derive meaningful insights.
Machine Learning (ML): Exploring algorithms and models that enable systems to learn from data.
Artificial Intelligence (AI): Delving into intelligent systems that mimic human behavior and decision-making.
Large Language Models (LLMs): Investigating advanced language processing technologies to enhance natural language understanding and generation.
I am eager to further develop my expertise in these fields and apply them to real-world applications, contributing to innovative projects and technologies."""

In [9]:
messages = []

In [85]:
user_input      = "yes"

In [86]:
probe_system_message = create_system_message("discuss", stage_config, mode='probe')
    
user_content         = f"""
    Here is what we know about the learner:
        {profile_summary}
           
    We need to probe the learner for the following fields: {required_fields}
    
    Learner's message: {user_input}
    """

In [87]:
message_payload = create_message_payload(user_content, 
                                             probe_system_message, 
                                             messages, 
                                             max_tokens=10000
                                             )

In [89]:
response_tool = force_tool_call(resp_model, 
                                    message_payload, 
                                    model='gpt-4o-mini',
                                    tool_choice='required',
                                    )

In [91]:
assistant_message = response_tool.message
Markdown(assistant_message)

Thank you for completing your profile, Mumtaz! 🎉 We're all set to start your personalized learning experience. Get ready to dive deep into statistics, machine learning, AI, and large language models over the next 6 weeks!  If you're excited, I can prepare some resources and materials tailored just for you. Let's make this learning journey a fantastic one! 🚀

In [92]:
messages.extend([{'role':'user','content':user_input},{'role':'assistant','content':assistant_message}])

In [93]:
messages

[{'role': 'user', 'content': 'hey'},
 {'role': 'assistant',
  'content': "Hey Mumtaz! 🌟 It's great to connect with you! I see you have some exciting interests. Let's dive a bit deeper to design the perfect learning plan for you! \n\nFirst, are there any particular areas within statistics, machine learning, AI, or LLMs that you're interested in exploring more? 🎯"},
 {'role': 'user',
  'content': 'statistics for learning bias or fairness in in models prediction, other interests also i would like to deeply learn from each of them'},
 {'role': 'assistant',
  'content': "Awesome focus on learning bias and fairness in model predictions! 🌟 Let's make sure we cover all your interests deeply. How do you prefer to learn? (e.g., videos, reading, hands-on activities) 🎬📚🛠️"},
 {'role': 'user', 'content': 'handon'},
 {'role': 'assistant',
  'content': "Fantastic! 🛠️ We'll incorporate hands-on activities into your learning. Now, let's discuss the curriculum! Here are some modules I was thinking about

In [94]:
extract_system_message = create_system_message('discover', stage_config, mode='extract')
conversation_summary=summarize_history(messages)

In [95]:
user_message = f"""The learner is at the **Discover** stage.
    This is the (unverified) information we have about the learner:
        {profile_summary}

    Here is the summary of the conversation:
        {conversation_summary}
                    
    Which of the following fields can we add or update about the learner?
        {', '.join(required_fields)}
    """

In [96]:
message_payload = create_message_payload(user_message,  extract_system_message, [], max_tokens=10000)

In [109]:
######### Creating pydantic model for field having multiple properties ##########

# for field in stage_config['fields']:
#     if 'properties' in field:
#         create_pydantic_model(field['title'], field['properties'], field['description'])

In [107]:
stage_config = get_yaml_data('discuss')

In [111]:
models = [create_pydantic_model(field['title'], field['properties'], field['description']) if 'properties' in field 
          else create_pydantic_model(field['title'], [field], field['description']) 
          for field in stage_config['fields']]


extraction_tools = force_tool_call(models,
                                       message_payload, 
                                       model='gpt-4o',
                                       tool_choice='required',
                                       parallel_tool_calls=True)

In [112]:
extraction_tools

[interest_areas(interest_areas='Statistics for learning bias or fairness in model predictions'),
 learning_style(learning_style='Kinesthetic'),
 Curriculum(Subject='AI and Machine Learning', Title='Comprehensive Learning Plan', Modules=['Introduction to Statistics and Data Analysis', 'Understanding Bias and Fairness in Machine Learning', 'Advanced Machine Learning Algorithms', 'Ethics in AI and Fairness Techniques', 'Deep Dive into Large Language Models']),
 timeline(timeline='6 weeks'),
 goals_alignment(goals_alignment='The plan aligns with goals')]

In [113]:
for tool in extraction_tools:
    print(tool.dict())

{'interest_areas': 'Statistics for learning bias or fairness in model predictions'}
{'learning_style': 'Kinesthetic'}
{'Subject': 'AI and Machine Learning', 'Title': 'Comprehensive Learning Plan', 'Modules': ['Introduction to Statistics and Data Analysis', 'Understanding Bias and Fairness in Machine Learning', 'Advanced Machine Learning Algorithms', 'Ethics in AI and Fairness Techniques', 'Deep Dive into Large Language Models']}
{'timeline': '6 weeks'}
{'goals_alignment': 'The plan aligns with goals'}


In [ ]:
# - field: prerequisites
  #   title: prerequisites
  #   annotation: list
  #   description: >
  #     The necessary qualifications or prior knowledge required for enrolling in the curriculum.
  #   probe_questions:
  #     - Do you have the necessary prerequisites for this curriculum? 📜
  #     - Would you like us to review any specific prerequisite knowledge? 🔍
  #   user_responses:
  #     - Yes, I meet all the prerequisites.
  #     - I might need a refresher on statistics.
  #   extract_examples:
  #     - "I might need a refresher on statistics > prerequisites='Needs refresher on statistics'"
  #     - "Yes, I meet all the prerequisites > prerequisites='Qualified to enroll'"
  #   probe_responses:
  #     - Great! You're ready to start. 🚀
  #     - No problem! We can provide resources for statistics. 📘

In [34]:
def parse_curriculum(curriculum_json):
    try:
        # Load the curriculum JSON
        curriculum_data = json.loads(curriculum_json)

        # Validate that the input is a list
        if not isinstance(curriculum_data, list) or not curriculum_data:
            raise ValueError("Expected a non-empty list of curriculums.")

        # Extract relevant fields for the first curriculum
        curriculum = curriculum_data[0]  # Assuming we want to process the first curriculum
        parsed_data = {
            "title": curriculum.get('title', '').strip(),
            "subject": curriculum.get('subject', '').strip(),
            "level": str(curriculum.get('level', '')).strip(),
            "modules": []
        }

        # Validate main curriculum fields
        if not parsed_data["title"]:
            raise ValueError("Curriculum title is required.")
        if not parsed_data["subject"]:
            raise ValueError("Curriculum subject is required.")
        if not parsed_data["level"].isdigit():
            raise ValueError("Curriculum level must be a numeric value.")

        # Loop through modules and extract information
        for module in curriculum.get('modules', []):
            module_info = {
                "module_title": module.get('title', '').strip(),
                "lessons": [],
                "learning_outcomes": []
            }

            # Validate module fields
            if not module_info["module_title"]:
                raise ValueError("Module title is required.")

            # Extract lessons within the module
            for lesson in module.get('lessons', []):
                lesson_title = lesson.get('title', '').strip()
                content = lesson.get('content', '').strip()
                duration = lesson.get('duration', 0)

                if not lesson_title:
                    raise ValueError("Lesson title is required.")
                if not content:
                    raise ValueError("Lesson content is required.")
                if not isinstance(duration, (int, float)) or duration <= 0:
                    raise ValueError(f"Lesson duration must be a positive number. Got: {duration}")

                module_info["lessons"].append({
                    "lesson_title": lesson_title,
                    "content": content,
                    "duration": duration
                })

            # Extract learning outcomes and assessment criteria
            for outcome in module.get('learning_outcomes', []):
                description = outcome.get('description', '').strip()
                assessment_criteria = outcome.get('assessment_criteria', [])

                if not description:
                    raise ValueError("Learning outcome description is required.")
                
                # Validate assessment criteria
                if not isinstance(assessment_criteria, list):
                    raise ValueError("Assessment criteria must be a list.")
                assessment_criteria = [criterion.strip() for criterion in assessment_criteria if criterion.strip()]

                module_info["learning_outcomes"].append({
                    "description": description,
                    "assessment_criteria": assessment_criteria
                })

            parsed_data["modules"].append(module_info)

        return parsed_data

    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return None
    except ValueError as ve:
        print(f"Validation error: {ve}")
        return None


In [35]:

curriculum_json = '''
[
 {
   "title": "Level 5 Diploma in Business Management",
   "subject": "Business Management",
   "level": "5",
   "modules": [
     {
       "title": "Organizational Behavior",
       "learning_outcomes": [
         {
           "description": "Understand and analyze organizational behavior concepts",
           "assessment_criteria": [
             "Explain key theories of organizational behavior",
             "Analyze the impact of organizational culture on performance"
           ]
         }
       ],
       "lessons": [
         {
           "title": "Introduction to Organizational Behavior",
           "content": "This lesson introduces the key concepts of organizational behavior...",
           "duration": 120
         },
         {
           "title": "Individual Behavior in Organizations",
           "content": "Learn about personality, perception, and individual decision making...",
           "duration": 120
         }
       ],
       "duration": 60
     }
   ],
   "prerequisites": ["Level 4 qualification in a related subject"],
   "qualification_level": 5,
   "guided_learning_hours": 240,
   "total_qualification_time": 480,
   "assessment_methods": ["Written examination", "Case study analysis", "Research project"]
 }
]
'''

In [36]:
parsed_curriculum = parse_curriculum(curriculum_json)
pprint.pprint(parsed_curriculum)


{'level': '5',
 'modules': [{'learning_outcomes': [{'assessment_criteria': ['Explain key '
                                                             'theories of '
                                                             'organizational '
                                                             'behavior',
                                                             'Analyze the '
                                                             'impact of '
                                                             'organizational '
                                                             'culture on '
                                                             'performance'],
                                     'description': 'Understand and analyze '
                                                    'organizational behavior '
                                                    'concepts'}],
              'lessons': [{'content': 'This lesson introduces the key concept